In [1]:
'''
Import Relevant Libraries
'''
import math
import time
import random
import numpy as np
from sympy import *

import ansatz
import ham_construction


import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14

from qiskit import *
from qiskit.providers.ibmq import least_busy

from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import Pauli, SparsePauliOp

from qiskit.circuit import Parameter
from qiskit.circuit import QuantumCircuit
from qiskit.circuit import ParameterVector

from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import FakeManila

from qiskit_algorithms.optimizers import SPSA, COBYLA, GradientDescent

from qiskit.primitives import Estimator as prim_Estimator
from qiskit_ibm_runtime import Estimator as run_Estimator
from qiskit_aer.primitives import Estimator as AerEstimator

from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

In [2]:
'''
Define IBMQ service and find list of backends.
'''
#service = QiskitRuntimeService()
#service.backends()

'\nDefine IBMQ service and find list of backends.\n'

In [3]:
def rand_numb(low_in,high_in):
    '''
    Returns a random number between low_in and high_in, including both end points.
    '''
    return np.random.uniform(low=low_in, high=np.nextafter(high_in, np.inf))

In [4]:
def costf(params, qCirc, hamiltonian, estimator, coeffI):
    '''
    Calculates the expectation value. 
    Inputs: 
        params: List of parameters
        qCirc: QuantumCircuit object of ansatz circuit
        hamiltonian: SparsePauliOp of the Hamiltonian
        estimator: Estimator object used to find cost function value
    '''
    job = estimator.run(qCirc, hamiltonian, params)
    result = job.result()
    return np.real(result.values[0]) + coeffI

In [5]:
def gradient(params, qCirc, hamiltonian, estimator):
    '''
    Calculates the gradient of the cost function. 
    Inputs: 
        params: List of parameters
        qCirc: QuantumCircuit object of ansatz circuit
        hamiltonian: SparsePauliOp of the Hamiltonian
        estimator: Estimator object used to find cost function value
    '''
    gradient = []
    for i in range(len(params)):
        param_plus = np.copy(params)
        param_minus = np.copy(params)
        param_plus[i] += np.pi/2
        param_minus[i] -= np.pi/2
        gradient.append(0.5*(np.real(estimator.run(qCirc, hamiltonian, param_plus).result().values[0]) - np.real(estimator.run(qCirc, hamiltonian, param_minus).result().values[0])))
        
    return np.array(gradient)

In [6]:
def callback(nfev, x, fx, stepsize, accepted = False):
    '''
    Callback function called automatically during optimization. Appends expectation value, current iteration and 
    prints every iteration.
    '''
    callback_dict["iters"] += 1
    callback_dict["cost_history"].append(fx)
    print(
        "Iters. done: {} Current Cost: {}".format(callback_dict["iters"], callback_dict["cost_history"][-1]),
        end="\r",
        flush=True,
    )

In [24]:
'''
Parameters for n+\alpha simulations. hOmega in MeV and b in fm*sqrt(MeV).
'''
hOmega = 12
lisAllCoeffs = [-57.2067, 6.6528, 0.08596, -0.013, -0.001, -1.7e-5, 2.4e-6, 2.1e-7, 5.7e-9, -3.7e-10, -4.4e-11, -1.6e-12, 4.7e-14]

#hOmega = 16
#lisAllCoeffs = [-59.5705, 6.4481, 0.1335, -0.006, -0.001, -4.4e-5, 3.6e-7, 2.2e-7, 1.5e-8, 3.1e-10, -3.4e-11, -3.7e-12, -1.6e-13]

lisAllCorrectedCoeffs = []

b = 6.442 * np.sqrt(5/4) * np.sqrt(1/hOmega)

for i in range(len(lisAllCoeffs)):
    lisAllCorrectedCoeffs.append(lisAllCoeffs[i]*math.pow(b, 2*i))

powerCutOff = 4
k = int(powerCutOff/2)
lisCoeffs = lisAllCorrectedCoeffs[0:k+1]
print(lisCoeffs)

[-57.2067, 28.759059251999997, 1.6063375909102695]


In [25]:
'''
Pick number of qubits and encoding type.
    encType = 1 for Gray code 
    encType = 2 for Binary Coding
    encType = 3 for One-Hot Coding
'''
numQubit = 4
encType = 1

In [26]:
'''
Create ansatz for the given encoding type and number of qubits.
'''
numLayer = 4
qCirc = ansatz.create_RealHE_Ansatz(numQubit, numLayer)

In [27]:
'''
Pick K-value for number of off-diagonals. Total number of diagonals is 2K+1. 
Create Hamiltonian for given parameters and K.
'''
hamiltonian_matrix, hamiltonian_operator, true_value = ham_construction.createHamiltonianAbInitio(encType, numQubit, hOmega, lisCoeffs)
reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
hamiltonian = SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs)
print("True Value : ", true_value)
hamiltonian_matrix

True Value :  -16.6190645599868


Matrix([
[0.955654843913507, 37.7108097275387,  4.3991366675503,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0],
[ 37.7108097275387, 94.5688372115675,  83.217724024215, 11.6390216057235,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0],
[  4.3991366675503,  83.217724024215, 207.458070670145, 141.414453881121,  22.083490805998,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0],
[                0, 11.6390216057235, 141.414453881121, 339.623355219645,  212.41543553066, 35.7387552344209,         

In [22]:
'''
Initialize random ansatz parameters. 
'''
paramProver = []
for i in range(0, len(qCirc.parameters)):
    paramProver = np.append(paramProver, np.array([rand_numb(0,2*np.pi)]))

In [23]:
def learning_rate_func():
    '''
    Defines learning rate function. Using a window size of 10, fit a straight line of the last 10 iterations. 
    If the slope is negative, increase learning rate lr to min(1.05*lr, lr_max).
    If the slope is positive, reduce learning rate lr to max(0.8*lr, lr_min).
    '''
    lr_min = 0.0001
    lr_max = 0.1
    def get_rate():
        lr = 0.002
        lr_wait = 0
        while True:
            lr_wait += 1
            if lr_wait >= 10:
                m, _ = np.polyfit(list(range(10)), callback_dict["cost_history"][-10:], 1)
                if m>0:
                    lr = max(0.8*lr, lr_min)
                else:
                    lr = min(1.05*lr, lr_max)
                lr_wait = 0
            yield lr
        
    return get_rate()

In [26]:
'''
Noiseless Simulation.
    1. Initiate empty callback_dict to track progress.
    2. Define estimator for noiseless simulation.
    3. Print initial cost.
    4. Wrap costf to implicitly inject the circuit, hamiltonian, and estimator.
    5. Train.
'''
callback_dict = {
    "iters": 0,
    "cost_history": [],
}
estimator = prim_Estimator()
noiselessParam = np.copy(paramProver)
count = 0
maxIterList = [500, 500, 1000]
resampList = [2, 2, 4]

for j in [0, 1, 2]:
    _, hamiltonian_operator, _ = ham_construction.createHamiltonianAbInitio(encType, numQubit, hOmega, lisAllCorrectedCoeffs[0:j+1])
    reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
    coeffI, hamiltonian = ham_construction.stripI(SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs), numQubit)
    
    print("Initial Cost : ", costf(noiselessParam, qCirc, hamiltonian, estimator, coeffI))

    autospsa = SPSA(maxiter=maxIterList[count], learning_rate=None, perturbation=None, callback=callback, resamplings=resampList[count])
    aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator, coeffI))
    obj = autospsa.minimize(fun=aug_costf, x0=noiselessParam)
    noiselessParam = obj.x
    count += 1

print("Initial Cost : ", costf(noiselessParam, qCirc, hamiltonian, estimator, coeffI))
gd = GradientDescent(maxiter=1000, learning_rate=learning_rate_func, tol=1e-07, callback=callback, perturbation=None)
aug_costf = GradientDescent.wrap_function(costf, (qCirc, hamiltonian, estimator, coeffI))
aug_gradient = GradientDescent.wrap_function(gradient, (qCirc, hamiltonian, estimator))
obj = gd.minimize(fun=aug_costf, x0=noiselessParam, jac=aug_gradient)
noiselessParam = obj.x

Initial Cost :  2.3547816892638522
Initial Cost :  259.13299874208842.924322122063635
Initial Cost :  -15.992173611178267.344668123331535
Initial Cost :  -16.108168212937017.108168212937017


In [27]:
noiseless_callback_dict = callback_dict

In [106]:
'''
Shot Noise Simulation.
    1. Initiate empty callback_dict to track progress.
    2. Define estimator for shot noise simulation.
    3. Print initial cost.
    4. Wrap costf to implicitly inject the circuit, hamiltonian, and estimator.
    5. Train.
'''
callback_dict = {
    "iters": 0,
    "cost_history": [],
}

estimator = prim_Estimator(options={"shots": 1e5})
shotNoiseParam = np.copy(paramProver)
count = 0
maxIterList = [500, 500, 500]
resampList = [2, 4]

for j in [0, 1, 2]:
    _, hamiltonian_operator, _ = ham_construction.createHamiltonianAbInitio(encType, numQubit, hOmega, lisAllCorrectedCoeffs[0:j+1])
    reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
    coeffI, hamiltonian = ham_construction.stripI(SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs), numQubit)

    print("Initial Cost : ", costf(shotNoiseParam, qCirc, hamiltonian, estimator, coeffI))
    
    autospsa = SPSA(maxiter=maxIterList[count], learning_rate=None, perturbation=None, callback=callback, resamplings=resampList[count])
    aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator, coeffI))
    obj = autospsa.minimize(fun=aug_costf, x0=shotNoiseParam)
    shotNoiseParam = obj.x
    
print("Initial Cost : ", costf(shotNoiseParam, qCirc, hamiltonian, estimator, coeffI))
gd = GradientDescent(maxiter=1500, learning_rate=learning_rate_func, tol=1e-07, callback=callback, perturbation=None)
aug_costf = GradientDescent.wrap_function(costf, (qCirc, hamiltonian, estimator, coeffI))
aug_gradient = GradientDescent.wrap_function(gradient, (qCirc, hamiltonian, estimator))
obj = gd.minimize(fun=aug_costf, x0=shotNoiseParam, jac=aug_gradient)
noiselessParam = obj.x

Initial Cost :  17.519750616249436
Initial Cost :  97.636462791923243.575452503243426
Initial Cost :  -10.351895972554757.937032184638383
Initial Cost :  2.09438334991341433100546381292595


In [107]:
shot_noise_callback_dict = callback_dict

In [117]:
'''
Noisy Simulation.
    1. Initiate empty callback_dict to track progress.
    2. Define estimator for noisy simulation.
    3. Print initial cost.
    4. Wrap costf to implicitly inject the circuit, hamiltonian, and estimator.
    5. Train.
'''
callback_dict = {
    "iters": 0,
    "cost_history": [],
}

device = FakeManila()
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
noisyParam = np.copy(paramProver)

estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "coupling_map": coupling_map,
        "noise_model": noise_model,
    },
    run_options={"shots": 1e3},
)

for j in [0, 1, 2]:
    _, hamiltonian_operator, _ = ham_construction.createHamiltonianAbInitio(encType, numQubit, hOmega, lisAllCorrectedCoeffs[0:j+1])
    reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
    coeffI, hamiltonian = ham_construction.stripI(SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs), numQubit)

    print("Initial Cost : ", costf(noisyParam, qCirc, hamiltonian, estimator, coeffI))

    autospsa = SPSA(maxiter=500, learning_rate=None, perturbation=None, callback=callback, resamplings=1)
    aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator, coeffI))
    obj = autospsa.minimize(fun=aug_costf, x0=noisyParam)
    noisyParam = obj.x


estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "coupling_map": coupling_map,
        "noise_model": noise_model,
    },
    run_options={"shots": 1e5},
)

_, hamiltonian_operator, _ = ham_construction.createHamiltonianAbInitio(encType, numQubit, hOmega, lisAllCorrectedCoeffs[0:3])
reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
coeffI, hamiltonian = ham_construction.stripI(SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs), numQubit)

print("Initial Cost : ", costf(noisyParam, qCirc, hamiltonian, estimator, coeffI))

autospsa = SPSA(maxiter=500, learning_rate=None, perturbation=None, callback=callback, resamplings=4)
aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator, coeffI))
obj = autospsa.minimize(fun=aug_costf, x0=noisyParam)
noisyParam = obj.x

print("Noiseless value on noisy trained parameters : ", costf(noisyParam, qCirc, hamiltonian, prim_Estimator(), coeffI))

Initial Cost :  9.841373645858809
Initial Cost :  156.90197303282224.146000567290035
Initial Cost :  226.88354116714572.17587370679287
Initial Cost :  222.578849524753884.09478677223683
Noiseless value on noisy trained parameters :  -14.871663642752083


In [118]:
noisy_trained_value = costf(noisyParam, qCirc, hamiltonian, prim_Estimator(), coeffI)
noisy_callback_dict = callback_dict

In [ ]:
LB = least_busy(service.backends(filters=lambda x: not x.configuration().simulator))
session = Session(service=service, backend=LB.name)
estimator = run_Estimator(session=session)
quantumParam = np.copy(paramProver)

print("Initial Cost : ", costf(quantumParam, qCirc, hamiltonian, estimator, coeffI))

In [ ]:
'''
Quantum Simulation.
    1. Initiate empty callback_dict to track progress.
    2. Define estimator for quantum simulation.
    3. Print initial cost.
    4. Wrap costf to implicitly inject the circuit, hamiltonian, and estimator.
    5. Train.
'''
callback_dict = {
    "iters": 0,
    "cost_history": [],
}

quan_options = Options(optimization_level=2, resilience_level=2)
quan_options.execution.shots = 4096
estimator = run_Estimator(session=session, options=quan_options)
quantumParam = np.copy(paramProver)

for j in range(1, k+1):
    if (j%2==0):
        continue
    print(j)

    _, hamiltonian_operator, _ = ham_construction.createHamiltonianExp(encType, numQubit, j, V0, c, hOmega)
    reversed_Pauli = [x[::-1].to_label() for x in hamiltonian_operator.paulis]
    hamiltonian = SparsePauliOp(reversed_Pauli, hamiltonian_operator.coeffs)
    
    print("Initial Cost : ", costf(quantumParam, qCirc, hamiltonian, estimator, coeffI))
    
    autospsa = SPSA(maxiter=500, learning_rate = None, perturbation=None, callback=callback, resamplings=1)
    aug_costf = SPSA.wrap_function(costf, (qCirc, hamiltonian, estimator, coeffI))
    obj = autospsa.minimize(fun=aug_costf, x0=quantumParam)
    quantumParam = obj.x
    
print("Noiseless value on quantum trained parameters : ", costf(quantumParam, qCirc, hamiltonian, prim_Estimator(), coeffI))

In [ ]:
quantum_callback_dict = callback_dict

In [119]:
list_parameter = [numQubit, encType, hOmega, powerCutOff] + lisCoeffs

parameter_string = ""
for item in list_parameter:
    parameter_string += str(item)
    parameter_string += " "

print(parameter_string)

4 1 12 4 -57.2067 28.759059251999997 1.6063375909102695 


In [120]:
def writeToFile(filename, parameter_string, qt_back_name):
    '''
    Write the data to a text file to be used to plot the data. The format is as follows:
        1. Type of file - Fixed numQubit, Fixed k, Different backends
        2. parameter_string = "V0 hOmega c numQubit encType k"
        3. num_noiseless
        4. loss_noiseless
        5. num_shot_noise
        6. loss_shot_noise
        7. num_noisy
        8. loss_noisy
        9. noisy_trained_value
    '''
    file = open(filename, "w+")
    
    file.write("Fixed numQubit, Fixed K, Different backends Ab-initio\n")
    #file.write("Quantum backend used : " + qt_back_name + "\n")
    
    file.write(parameter_string + "\n")
    
    file.write(str(true_value) + "\n")
    
    file.write(str(noiseless_callback_dict["iters"]) + "\n")
    L = [str(i) + "\n" for i in noiseless_callback_dict["cost_history"]]
    file.writelines(L)
    
    file.write(str(shot_noise_callback_dict["iters"]) + "\n")
    L = [str(i) + "\n" for i in shot_noise_callback_dict["cost_history"]]
    file.writelines(L)
    
    file.write(str(noisy_callback_dict["iters"]) + "\n")
    L = [str(i) + "\n" for i in noisy_callback_dict["cost_history"]]
    file.writelines(L)
    file.write("Noisy Trained Value : " + str(noisy_trained_value))
    
    #L = [str(i) + "\n" for i in quantum_callback_dict["cost_history"]]
    #file.writelines(L)

    file.close()
    
writeToFile("Data/ABI_" + str(hOmega) + "_n_"+str(numQubit)+"_k_"+str(powerCutOff)+"-"+time.strftime("%Y%m%d-%H%M%S"), parameter_string, "LB.name")